In [92]:
%load_ext autoreload
%autoreload 2

import tensorflow_quantum as tfq
from utilities.variational import VQE
from utilities.qmodels import *
from utilities.evaluator import Evaluator
from utilities.idinserter import IdInserter
from utilities.unitary_killer import UnitaryMurder
from utilities.simplifier import Simplifier
import cirq

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [93]:
vqe_handler = VQE(n_qubits=2, problem_config={"problem":"XXZ","g":1,"J":2})
iid_inserter = IdInserter(n_qubits = vqe_handler.n_qubits, selector_temperature=0, epsilon=0.1)
simplifier = Simplifier(n_qubits=vqe_handler.n_qubits,testing=True)

In [94]:
Mindexed_circuit = [vqe_handler.number_of_cnots+vqe_handler.n_qubits,vqe_handler.number_of_cnots+vqe_handler.n_qubits + 1]+ iid_inserter.resolution_2cnots(0,1)+[vqe_handler.number_of_cnots+vqe_handler.n_qubits + 1]

In [95]:
vqe_handler.give_circuit(Mindexed_circuit)[0]

(0, 0): ───Rx(th_0)───@───Rz(th_2)───Rx(th_3)───Rz(th_4)───@──────────────
                      │                                    │
(0, 1): ───Rx(th_1)───X───Rx(th_5)───Rz(th_6)───Rx(th_7)───X───Rx(th_8)───

In [110]:
a,b,Mindex_to_symbols = vqe_handler.give_circuit(Mindexed_circuit)
Menergy, OMresolver, history = vqe_handler.vqe(Mindexed_circuit)

In [129]:
OMresolver

{'th_0': -12.56637,
 'th_1': 24.393335,
 'th_2': 7.7470903,
 'th_3': 4.712301,
 'th_4': 32.986675,
 'th_5': -19.103405,
 'th_6': -3.1415343,
 'th_7': 18.186104,
 'th_8': 9.094963}

In [128]:
OMresolver["th_5"]+OMresolver["th_7"]

-0.9173012

In [178]:
vqe_handler.give_circuit(Mindexed_circuit)[0]

(0, 0): ───Rx(th_0)───@───Rz(th_2)───Rx(th_3)───Rz(th_4)───@──────────────
                      │                                    │
(0, 1): ───Rx(th_1)───X───Rx(th_5)───Rz(th_6)───Rx(th_7)───X───Rx(th_8)───

In [179]:
vqe_handler.give_unitary(Mindexed_circuit,OMresolver)

(0, 0): ───Rx(0)─────────@───Rz(-1.53π)───Rx(1.5π)───Rz(-1.5π)───@───────────────
                         │                                       │
(0, 1): ───Rx(-0.235π)───X───Rx(1.92π)────Rz(-π)─────Rx(1.79π)───X───Rx(-1.1π)───

In [207]:
Sindexed_circuit, Sresolver, Sindex_to_symbols = simplifier.simplify_step(Mindexed_circuit, OMresolver, Mindex_to_symbols)

connections
{'0': ['rx', 0, 'rz', 'rx', 'rz', 0], '1': ['rx', 0, 'rx', 'rz', 'rx', 0, 'rx']}


places_gates:
{'0': [0, 2, 3, 4, 5, 9], '1': [1, 2, 6, 7, 8, 9, 10]}


******
{'th_0': -12.56637, 'th_1': 7.7470903, 'th_2': 4.712301, 'th_3': 32.986675}
[4, 5, 0, 2, 4, 2, 5, 3, 5, 0, 5]
{'0': ['th_0', 'th_1', 'th_2', 'th_3'], '1': []}
7
['th_5']
7
['th_5', 'th_8']
******
{'th_0': -12.56637, 'th_1': 7.7470903, 'th_2': 4.712301, 'th_3': 32.986675, 'th_4': 5.2899303, 'th_5': -19.103405, 'th_6': -3.1415343, 'th_7': 27.281067, 'th_8': 9.094963}
[4, 5, 0, 2, 4, 2, -1, 3, 5, 0, -1]
{'0': ['th_0', 'th_1', 'th_2', 'th_3'], '1': ['th_4', 'th_5', 'th_6', 'th_7', 'th_8']}


In [206]:
vqe_handler.give_circuit(Mindexed_circuit)[0]

(0, 0): ───Rx(th_0)───@───Rz(th_2)───Rx(th_3)───Rz(th_4)───@──────────────
                      │                                    │
(0, 1): ───Rx(th_1)───X───Rx(th_5)───Rz(th_6)───Rx(th_7)───X───Rx(th_8)───

In [208]:
OMresolver

{'th_0': -12.56637,
 'th_1': 24.393335,
 'th_2': 7.7470903,
 'th_3': 4.712301,
 'th_4': 32.986675,
 'th_5': -19.103405,
 'th_6': -3.1415343,
 'th_7': 18.186104,
 'th_8': 9.094963}

In [187]:
dicti = {'th_0': -12.56637, 'th_1': 7.7470903, 'th_2': 4.712301, 'th_3': 32.986675, 'th_4': 5.2899303, 'th_5': -19.103405, 'th_6': -3.1415343, 'th_7': 27.281067, 'th_8': 9.094963}

In [189]:
dicti.pop("th_8")
dicti.pop("th_5")

-19.103405

In [192]:
dd={'th_0': -12.56637,
 'th_1': 7.7470903,
 'th_2': 4.712301,
 'th_3': 32.986675,
 'th_4': 5.2899303,
 'th_5': -3.1415343,
 'th_6': 27.281067}

In [196]:
nn = [4, 5, 0, 2, 4, 2, -1, 3, 5, 0, -1]

In [199]:
for ind, k in enumerate(nn):
    if k == -1:
        nn.pop(ind)

In [203]:
cirq.resolve_parameters(vqe_handler.give_circuit(nn)[0],dd)

(0, 0): ───Rx(0)────────@───Rz(1.5π)───Rx(-1.5π)────Rz(1.68π)───@───
                        │                                       │
(0, 1): ───Rx(-1.53π)───X───Rz(-π)─────Rx(0.684π)───────────────X───

In [150]:
vqe_handler.give_circuit(Sindexed_circuit)[0]

(0, 0): ───Rx(th_0)───@───Rz(th_2)───Rx(th_3)───Rz(th_4)───@───
                      │                                    │
(0, 1): ───Rx(th_1)───X───Rz(th_5)───Rx(th_6)──────────────X───

In [135]:
OMresolver

{'th_0': -12.56637,
 'th_1': 24.393335,
 'th_2': 7.7470903,
 'th_3': 4.712301,
 'th_4': 32.986675,
 'th_5': -19.103405,
 'th_6': -3.1415343,
 'th_7': 18.186104,
 'th_8': 9.094963}

In [143]:
vqe_handler.give_unitary(Mindexed_circuit,OMresolver)

(0, 0): ───Rx(0)─────────@───Rz(-1.53π)───Rx(1.5π)───Rz(-1.5π)───@───────────────
                         │                                       │
(0, 1): ───Rx(-0.235π)───X───Rx(1.92π)────Rz(-π)─────Rx(1.79π)───X───Rx(-1.1π)───

In [142]:
vqe_handler.give_unitary(indexed_circuit,resolver)

NameError: name 'indexed_circuit' is not defined

In [120]:
u1 = vqe_handler.give_unitary(Mindexed_circuit,OMresolver)
u1

(0, 0): ───Rx(0)─────────@───Rz(-1.53π)───Rx(1.5π)───Rz(-1.5π)───@───────────────
                         │                                       │
(0, 1): ───Rx(-0.235π)───X───Rx(1.92π)────Rz(-π)─────Rx(1.79π)───X───Rx(-1.1π)───

In [125]:
u2 = vqe_handler.give_unitary(Sindexed_circuit,Sresolver)
u2

(0, 0): ───Rx(0)───────@───Rz(-1.53π)───Rx(1.5π)───Rz(-1.5π)───@───
                       │                                       │
(0, 1): ───Rx(1.68π)───X───Rz(1.92π)────Rx(-π)─────────────────X───

In [47]:
cirq.unitary(u1)-cirq.unitary(u2)

array([[ 0.22822967+0.42700392j, -0.3101406 +0.93513739j,
         0.65801768-0.73328303j, -0.03857048-0.48263577j],
       [-0.65805645-0.73323178j, -0.0385391 +0.48263224j,
         0.22820516-0.42702385j,  0.31020434+0.93512915j],
       [ 0.65802092+0.73328476j,  0.03856339-0.48263342j,
         0.22822411-0.42699894j,  0.31014426+0.93513768j],
       [-0.2281996 -0.42701886j,  0.310208  -0.93512944j,
         0.65805969-0.73323351j, -0.03853201-0.48262989j]])

In [48]:
Sindexed_circuit, Sresolver, Sindex_to_symbols = simplifier.reduce_circuit(Mindexed_circuit, OMresolver, Mindex_to_symbols)

In [49]:
u2 = vqe_handler.give_unitary(Sindexed_circuit,Sresolver)
u2

(0, 0): ───Rx(2π)──────@───Rz(1.63π)─────Rx(0.5π)───Rz(1.5π)───@───
                       │                                       │
(0, 1): ───Rx(0.48π)───X───Rz(-0.182π)───Rx(-π)────────────────X───

In [50]:
simplifier.testing

False